In [1]:
import torch
from PIL import Image
import numpy as np
import glob
import yaml
from utils import *
import time
import shutil
import torch.backends.cudnn as cudnn
from test_dataloader import get_loader_test
import pickle
import argparse
from PLIPmodel import Create_PLIP_Model
from visual_model import *
import math
device = "cuda"


/home/toonies/anaconda3/envs/UTE-AI/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from PLIPmodel import Create_PLIP_Model

model = Create_PLIP_Model().to(device)
checkpoint = torch.load("checkpoints/PLIP_RN50.pth.tar",map_location='cpu')
model.image_encoder.load_state_dict(checkpoint["ImgEncoder_state_dict"])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
des_path =  "data/PLIP_Features/"
paths = "data/KeyFramesC00_V00/"
video_paths = sorted(glob.glob(f"{paths}/*/"))
video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

In [4]:
import torchvision.transforms as transforms

transform = transforms.Compose([
            transforms.Resize((256,128), interpolation=3),
            transforms.ToTensor(),
            transforms.Normalize((0.357, 0.323, 0.328),
                                    (0.252, 0.242, 0.239))
])

for vd_path in video_paths:
    print(vd_path)
    re_feats = []
    keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
    keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))
    for keyframe_path in keyframe_paths:
        model = model.eval()
        image = Image.open(keyframe_path).convert('RGB')
        image = transform(image).to(device)
        image = image.unsqueeze(0)
        with torch.no_grad():
            image_feats= model.get_image_embeddings(image)
            
        image_feats /= image_feats.norm(dim=-1, keepdim=True)
        image_feats = image_feats.detach().cpu().numpy().astype(np.float16).flatten() 

        re_feats.append(image_feats)

    name_npy = vd_path.split('/')[-1]
    outfile = f'{des_path}/{name_npy}.npy'
    np.save(outfile, re_feats) 

/home/toonies/anaconda3/envs/UTE-AI/lib/python3.7/site-packages/torchvision/transforms/transforms.py:330: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


data/KeyFramesC00_V00/C00_V0000
data/KeyFramesC00_V00/C00_V0001


KeyboardInterrupt: 